# 📈 Tablas Dinámicas en Pandas
Las **tablas dinámicas** (o *pivot tables*) en **Pandas** son una herramienta poderosa para **resumir, reorganizar y analizar datos tabulares** de forma flexible. Se implementan principalmente mediante la función [`pandas.pivot_table()`](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html), aunque también se pueden usar `groupby()`, `crosstab()` o `melt()` según el caso.

### Características principales de las tablas dinámicas:
- **Resumen de datos**: Las tablas dinámicas pueden resumir rápidamente grandes conjuntos de datos calculando sumas, promedios, conteos y otras funciones agregadas.
- **Reorganización dinámica**: Los usuarios pueden arrastrar y soltar campos fácilmente para reorganizar el diseño de la tabla dinámica, lo que permite distintas perspectivas sobre los datos (convertir filas en columnas y viceversa).
- **Filtrado y ordenamiento**: Las tablas dinámicas permiten filtrar y ordenar los datos para enfocarse en información específica o identificar tendencias.
- **Agrupación de datos**: Los usuarios pueden agrupar datos por categorías, fechas o rangos numéricos para crear resúmenes más significativos.
- **Campos calculados**: Es posible crear cálculos personalizados dentro de la tabla dinámica para obtener nuevas ideas a partir de los datos.
- **Explorar relaciones** entre múltiples variables.
- **Preparar datos para visualización** o reporting.

### Casos de uso comunes:
- Análisis de ventas por región, producto o período de tiempo.
- Informes financieros y presupuestación.
- Segmentación de clientes y análisis de comportamiento.
- Gestión y seguimiento de inventario.

## ✅ Requisitos previos

Instalamos lo necesario:
```bash
pip install pandas seaborn
```


## 📊 Ejemplos con datasets reales

### 🔹 Ejemplo 1: Resumen básico — Promedio de propinas por día (dataset `tips`)

In [17]:
import pandas as pd
import seaborn as sns


In [33]:
# **Objetivo**: Ver el promedio de propina por día de la semana.
df = pd.read_csv('../data/tips.csv')
pt = pd.pivot_table(df, values='tip', index='day', aggfunc='mean')
print(pt)

           tip
day           
Fri   2.734737
Sat   2.993103
Sun   3.255132
Thur  2.771452


🔹 Ejemplo 2: Dos dimensiones — Propina promedio por día y género

**Objetivo**: Comparar propinas promedio por día y sexo del cliente.

In [22]:
pt = pd.pivot_table(df, values='tip', index='day', columns='sex', aggfunc='mean')
print(pt)

sex     Female      Male
day                     
Fri   2.781111  2.693000
Sat   2.801786  3.083898
Sun   3.367222  3.220345
Thur  2.575625  2.980333


🔹 Ejemplo 3: Múltiples métricas — Conteo y suma de facturas (`total_bill`)

In [34]:
pt = pd.pivot_table(
    df,
    values='total_bill',
    index='day',
    aggfunc={'total_bill': ['sum', 'count']}
)
print(pt)

      count      sum
day                 
Fri      19   325.88
Sat      87  1778.40
Sun      76  1627.16
Thur     62  1096.33


> ⚠️ Nota: En versiones recientes de Pandas, `aggfunc` como dict puede requerir `margins=False` o ajustes si hay ambigüedad.

Alternativa más clara:

```python
pt = df.groupby('day')['total_bill'].agg(['sum', 'count'])
```

Pero para mantener el estilo tabla dinámica:

```python
pt = pd.pivot_table(
    df,
    values='total_bill',
    index='day',
    aggfunc=['sum', 'count']
)
```

🔹 Ejemplo 4: Relleno de valores faltantes y márgenes

In [24]:
# **Objetivo**: Ver promedio de propina por día y momento (almuerzo/cena), incluyendo totales.
pt = pd.pivot_table(
    df,
    values='tip',
    index='day',
    columns='time',
    aggfunc='mean',
    fill_value=0,
    margins=True  # Total general
)
print(pt)

time    Dinner     Lunch       All
day                               
Fri   2.940000  2.382857  2.734737
Sat   2.993103  0.000000  2.993103
Sun   3.255132  0.000000  3.255132
Thur  3.000000  2.767705  2.771452
All   3.102670  2.728088  2.998279


🔹 Ejemplo 5: Dataset de vuelos

Retrasos promedio por aerolínea y mes (`flights`)

In [35]:
# **Objetivo**: Ver pasajeros por año y mes en formato de tabla cruzada.
flights = pd.read_csv('../data/flights.csv')
# Este dataset tiene columnas: year, month, passengers

pt = pd.pivot_table(
    flights,
    values='passengers',
    index='year',
    columns='month',
    aggfunc='sum'
)
print(pt.head())

month  April  August  December  February  January  July  June  March  May  \
year                                                                        
1949     129     148       118       118      112   148   135    132  121   
1950     135     170       140       126      115   170   149    141  125   
1951     163     199       166       150      145   199   178    178  172   
1952     181     242       194       180      171   230   218    193  183   
1953     235     272       201       196      196   264   243    236  229   

month  November  October  September  
year                                 
1949        104      119        136  
1950        114      133        158  
1951        146      162        184  
1952        172      191        209  
1953        180      211        237  


🔹 Ejemplo 6: Análisis de ventas — Dataset de órdenes públicas

Usamos un subset del famoso dataset **"Sample - Superstore"**.  

In [28]:

df_super = pd.read_csv('../data/Superstore.csv', encoding='latin1')

# Aseguramos que Order Date sea datetime
df_super['Order Date'] = pd.to_datetime(df_super['Order Date'])

# Creamos columna de año
df_super['Order Year'] = df_super['Order Date'].dt.year

# Tabla dinámica: Ventas totales por categoría y año
pt = pd.pivot_table(
    df_super,
    values='Sales',
    index='Category',
    columns='Order Year',
    aggfunc='sum',
    fill_value=0
)
print(pt)

Order Year              2019        2020        2021         2022
Category                                                         
Furniture        157192.8531  170518.237  198901.436  215387.2692
Office Supplies  151776.4120  137233.463  183939.982  246097.1750
Technology       175278.2330  162780.809  226364.180  271730.8110


🔹 Ejemplo 7: Múltiples valores y funciones personalizadas

**Objetivo**: Analizar ventas y ganancias por categoría y subcategoría de productos, aplicando funciones personalizadas para obtener insights más detallados.

In [36]:
# Usamos el mismo df_super
# **Objetivo**: Obtener promedio, suma de ventas y coeficiente de variación de ganancias por región y segmento.
# Esto genera una tabla con **multi-índices** en columnas, muy útil para análisis avanzado.
def coef_var(x):
    return x.std() / x.mean() if x.mean() != 0 else 0

pt = pd.pivot_table(
    df_super,
    values=['Sales', 'Profit'],
    index='Region',
    columns='Segment',
    aggfunc={
        'Sales': ['mean', 'sum'],
        'Profit': [coef_var, 'mean']
    },
    fill_value=0
)
print(pt)

            Profit                                                           \
          coef_var                              mean                          
Segment   Consumer  Corporate Home Office   Consumer  Corporate Home Office   
Region                                                                        
Central  39.190058  13.422264    5.499809   7.066046  27.791831   28.398202   
East     10.033071   6.397258    5.160007  28.040153  26.935666   53.205611   
South     6.428529   8.024026   10.415527  32.116435  29.833771   16.987626   
West      5.520473   3.337098    6.965925  34.360409  35.872323   28.949939   

              Sales                                                   \
               mean                                 sum                
Segment    Consumer   Corporate Home Office    Consumer    Corporate   
Region                                                                 
Central  207.946728  234.763466  208.248046  252031.434  157995.8128   
East   

🔹 Ejemplo 8: Tabla dinámica + agrupamiento temporal + cálculo de crecimiento (%)

Usamos datos de pasajeros aéreos (`flights`), y calculamos **crecimiento anual por mes**.

**Objetivo**: Ver el porcentaje de crecimiento mensual año tras año.  

Esto combina:
- `pivot()` para reestructurar,
- cálculo personalizado (`pct_change`),
- `melt()` para volver a formato largo,
- `pivot_table()` final para presentación.

In [31]:
# Pivot original
pivot = flights.pivot(index='year', columns='month', values='passengers')

# Calculamos crecimiento anual (%)
growth = pivot.pct_change(periods=1) * 100  # comparado con año anterior

# Volvemos a formato largo si queremos tabla dinámica clásica
growth_long = growth.reset_index().melt(
    id_vars='year',
    var_name='month',
    value_name='growth_pct'
).dropna()

# Tabla dinámica del crecimiento
pt_growth = pd.pivot_table(
    growth_long,
    values='growth_pct',
    index='month',
    columns='year',
    fill_value=0
)
print(pt_growth.round(1))

year       1950  1951  1952  1953  1954  1955  1956  1957  1958  1959  1960
month                                                                      
April       4.7  20.7  11.0  29.8  -3.4  18.5  16.4  11.2   0.0  13.8  16.4
August     14.9  17.1  21.6  12.4   7.7  18.4  16.7  15.3   8.1  10.7   8.4
December   18.6  18.6  16.9   3.6  13.9  21.4  10.1   9.8   0.3  20.2   6.7
February    6.8  19.0  20.0   8.9  -4.1  23.9  18.9   8.7   5.6   7.5  14.3
January     2.7  26.1  17.9  14.6   4.1  18.6  17.4  10.9   7.9   5.9  15.8
July       14.9  17.1  15.6  14.8  14.4  20.5  13.5  12.6   5.6  11.6  13.5
June       10.4  19.5  22.5  11.5   8.6  19.3  18.7  12.8   3.1   8.5  13.3
March       6.8  26.2   8.4  22.3  -0.4  13.6  18.7  12.3   1.7  12.2   3.2
May         3.3  37.6   6.4  25.1   2.2  15.4  17.8  11.6   2.3  15.7  12.4
November    9.6  28.1  17.8   4.7  12.8  16.7  14.3  12.5   1.6  16.8   7.7
October    11.8  21.8  17.9  10.5   8.5  19.7  11.7  13.4   3.5  13.4  13.3
September  1

## ✅ Conclusión

Las tablas dinámicas en Pandas:
- Permiten **agregaciones flexibles**.
- Soportan **múltiples dimensiones y métricas**.
- Se integran con el ecosistema de **limpieza y visualización** (Matplotlib, Seaborn).
- Son ideales para **análisis exploratorio rápido** y **reportes estructurados**.

Con datasets reales como `tips`, `flights`, o el de *Superstore*, puedes aplicar estas técnicas a problemas de negocios, logística, marketing o finanzas.

> 📌 **Tip**: Usa `df.pivot_table()` para análisis cruzados, y combínalo con `groupby()` cuando necesites lógica más compleja antes del pivoteo.